In [2]:
import carla 
import math 
import random 
import time 
import numpy as np
import cv2

# Connect the client and set up bp library and spawn points
client = carla.Client('localhost', 2000) 
world = client.get_world()
bp_lib = world.get_blueprint_library()  
spawn_points = world.get_map().get_spawn_points() 

In [3]:
for v in world.get_actors().filter('*vehicle*'):
    v.destroy()
for s in world.get_actors().filter('*sensor*'):
    s.destroy()

In [4]:
# Spawn ego vehicle
vehicle_bp = bp_lib.find('vehicle.tesla.model3') 
vehicle = world.try_spawn_actor(vehicle_bp, random.choice(spawn_points))

# Move spectator behind vehicle to view
spectator = world.get_spectator() 
transform = carla.Transform(vehicle.get_transform().transform(carla.Location(x=-4,z=2.5)),vehicle.get_transform().rotation) 
spectator.set_transform(transform)

In [5]:
import importlib
import sys
from car.sensors import CarEquipment, base_sensor_specification, model3_base_offsets, base_sensor_types, base_callbacks, top_offsets, top_callbacks, over_offsets, over_callbacks

In [50]:
 #import the module here, so that it can be reloaded.
importlib.reload(sys.modules['car.sensors'])
from car.sensors import CarEquipment, base_sensor_specification, model3_base_offsets, base_sensor_types, base_callbacks, top_offsets, top_callbacks, over_offsets, over_callbacks

In [37]:
equipment = CarEquipment(world, vehicle)

In [8]:
equipment.build_sensors('base', base_sensor_types[1], base_sensor_specification, model3_base_offsets)

In [9]:
equipment.base_sensors['front'].is_listening

False

In [18]:
equipment.toggle_sensors('base', 'stop', equipment.rgb_callback, base_callbacks)

In [11]:
img = equipment.sensor_data['rear']

In [12]:
def layout_func(sensor_data):
    row1 = np.concatenate([sensor_data['front_left'], sensor_data['front'], sensor_data['front_right']], axis=1)
    row2 = np.concatenate([sensor_data['side_left'], sensor_data['rear'], sensor_data['side_right']], axis=1)
    tiled = np.concatenate([row1,row2], axis=0)
    return tiled

In [13]:
equipment.display_sensors('Base RGB', layout_func, equipment.sensor_data)

In [27]:
equipment.build_sensors('top', base_sensor_types[1], base_sensor_specification, top_offsets)

In [28]:
equipment.toggle_sensors('top', 'start', equipment.rgb_callback, top_callbacks)

In [29]:
def top_layout(sensor_data):
    return sensor_data['hd1']

In [30]:
equipment.display_sensors('Top RGB', top_layout, equipment.sensor_data)

In [132]:
vehicle.apply_control(carla.VehicleControl(throttle=.5, steer=-0.0, brake=0.1, 
                                           hand_brake=False, reverse=False, 
                                           manual_gear_shift=False, gear=0))

In [13]:
x = np.random.rand(100,32,32,3)

In [16]:
np.roll(x,2,-1).shape

(100, 32, 32, 3)

In [59]:
equipment.build_sensors('base', base_sensor_types[0], base_sensor_specification, over_offsets)

In [60]:
equipment.toggle_sensors('base', 'start', equipment.sem_callback, over_callbacks)

In [51]:
def layout_func(sensor_data):
    row1 = np.concatenate([sensor_data['o1'], sensor_data['o2'], sensor_data['o3']], axis=1)
    row2 = np.concatenate([sensor_data['o4'], sensor_data['o5'], sensor_data['o6']], axis=1)
    tiled = np.concatenate([row1,row2], axis=0)
    return tiled

In [62]:
equipment.display_sensors('Over Semantic', layout_func, equipment.sensor_data)

In [57]:
equipment.sensor_data.keys()

dict_keys(['1', 'o5', 'o4', 'o3', 'o2', 'o1', 'o6'])